## Description
This is a notebook for debugging the quantity in the amiral code for both the python and the IDL version. 

At the moment, the IDL version is working properly but we do need to check the defintion for the Fourier variables (regarding fft shift and fft transform). We need to make sure that we are not being an idiot ;) 

# Section
<details>
  <summary>Click to expand!</summary>
  
  ## Heading
  1. [Generate a PSF](#Generate-a-PSF)
    * [Zero Padding](#Zero-Padding)
    * [Setting up the system](#Setting-up-the-system)
    * [Pupil Function](#Pupil-Function)
    
  2.[Minimisation](#Minimisation)
    
  2. [Doesnt match with the IDL output](#Doesnt-match)
  
</details>

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
SEED = -1234       # Seed for random numbers generation
FLUX = 5e8         # Object total intensity [e-]
READ_OUT_NOISE = 10. # CCD read-out noise standard-deviation [e-]
DATA_DIR = "/Users/alau/IDLWorkspace/Data/Vesta_OASIS/"
DATA_FITS = DATA_DIR + "2018-06-08T05_27_05.809.fits"
DIMENSION = 512 # Dimension of the image
LOOP = True

In [ ]:
# Setup of the system
fits_img = fits.open(DATA_FITS)
fits_img.info() 

obj = fits_img[0].data
obj_resize = np.zeros((DIMENSION,DIMENSION))

cuta = DIMENSION//2-128
cutb = DIMENSION//2+128

obj_resize[cuta:cutb,cuta:cutb] = obj

# Calibrating the flux
obj_resize = obj_resize/np.sum(obj_resize)*FLUX

aosys = instructment.aoSystem(sampling = 4,diameter = 7., occ_ratio = 0.1, no_acutuator = 30, wavelength = 500, dimension = DIMENSION)
fX, fY, freqnull = aosys.psd_frequency_array(DIMENSION)

pupil = aosys.get_pupil_plane(DIMENSION)

otf_tel = aosys.pupil_to_otf_tel(pupil,0*pupil, over_sampling=4)
psf_tel = np.abs(utils.ifft2D(otf_tel))


param = {
    "r0": 0.12,                  
    "background": 1e-10 ,      
    "amplitude": 1.8,       
    "ax": 0.05,                            
    "ay": 0.05, 
    "theta": 0., 
    "beta": 1.5,
    "mu": 1., 
    "rho0": 1., 
    "p": 1.
}

param_key, param_input = utils.dict2array(param)

psd_ao = aosys.psd_residual_ao (guess = param_input)
psd_halo = aosys.psd_residual_halo(r0=0.15)

psd = psd_halo + psd_ao 

integral, SR = aosys.psd_integral(psd, r0=param["r0"])
otf_atmo = aosys.otf_atmo(psd)
otf_total = aosys.otf_total(otf_tel, otf_atmo)
psf_total = aosys.psfao(otf_total)

print("\nSum of the PSF (which should be excatly 1.)", np.sum(np.abs(psf_total)))
print("\nMax of the otf_atmo: (which should be excatly 1.)", np.max(otf_atmo))
print("\nMax of the otf_tel: (which should be excatly 1.)", np.max(otf_tel))
print("\nMax of the OTF (which should be excatly 1.)", np.max(otf_total))

In [ ]:
# Strehl's ratio

psf_diffraction = aosys.psfao(otf_tel)

print("\nSR (PSF ratio): ", np.max(psf_total)/ np.max(psf_diffraction))
print("\nSR from the integral: ", SR)

In [ ]:
rcParams['figure.figsize'] = 13 ,11
fig, ax = plt.subplots(1,3)
fig.tight_layout(pad=0.4, w_pad=0.6, h_pad=4.0)

ycent = 512//2
ax[0].plot(otf_total[ycent, :])
ax[0].set_title('OTF(total)')
ax[0].axhline(y=1, color = 'r', ls = '--')

ax[1].plot(otf_tel[ycent, :])
ax[1].set_title('OTF(telescope)')
ax[1].axhline(y=1, color = 'r', ls = '--')

ax[2].plot(otf_atmo[ycent, :])
ax[2].set_title('OTF(atmosphere)')
ax[2].axhline(y=1, color = 'r', ls = '--')

In [ ]:
ft_obj = utils.fft2D(obj_resize,norm = False)
ft_image = ft_obj*otf_total

RON = 10. 

# conv_image = utils.ifft2D (ft_image, norm = True) + RON*np.random.randn(DIMENSION,DIMENSION)
conv_image = np.real(utils.ifft2D (ft_image, norm = False))


# Import the Data from IDL
fits_img = fits.open("/Users/alau/Repo/amiral/example/image.fits")
fits_img.info() 

conv_image = fits_img[0].data
print(np.max(conv_image), np.min(conv_image))

## AMIRAL guess

In [ ]:
amiral_guess = {
    "r0": 0.12,                  
    "background": 1e-10 ,      
    "amplitude": 3.,       
    "ax": 0.05,                            
    "ay": 0.05, 
    "theta": 0., 
    "beta": 1.5, 
    "mu": 1., 
    "rho0": 2., 
    "p": 2.9
}

amiral_guess = param 
amiral_guess["r0"] = 0.12
amiral_guess["mu"] = 1.
amiral_guess["rho0"] = 2.
amiral_guess["p"] = 3.

psf_param, guess = utils.dict2array(amiral_guess)


guess

In [ ]:
#lowerbound = param_min,[1D-6,rho0norm_min,power_bounds_in[0]]*norm
#upperbound = param_max,[FLT_MAX, 10D*dim, power_bounds_in[1]]*norm

FLT_MAX = 3.4028235e+38 # limit in IDL

# param = [r0, bck, sig, ax, ay, theta, beta]

# Should have fixed 
param_min = np.asarray([0.1, 1e-10 , 3.,0.05, 0.05, 0., 1.5])
param_max =  np.asarray([1.,2e-10, 3., 0.05 , 0.05,0. , 1.5])

#power_bounds_in =  np.asarray([1e-4, 4.5])

rho0norm_min = 1e-6

norm = np.array((amiral_guess["mu"],amiral_guess["rho0"],1.))
print("\nNormalisation constant: ", norm)


hyper_min =  np.asarray([0., 2., 2.9])
hyper_max =  np.asarray([1., 2. ,3.])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

psf_param, psf_guess = utils.dict2array(amiral_guess)

fixed_psf = psf_guess

norm = np.array((amiral_guess["mu"],amiral_guess["rho0"],1.))

rho0norm_min = 1e-6
fixed_var = psf_guess
print('lenght',len(fixed_var))

## Minimisation

In [ ]:
print(psf_guess)

In [ ]:
amiral_child = parameter.amiral(img=conv_image, guess=psf_guess, aosys = aosys, grad = None, upperbound = upperbound, lowerbound= lowerbound)
amiral_child.varible4criterion(psf_guess)
est_psf_param, value_criterion, value_grad = amiral_child.minimisation(psf_guess)

In [ ]:
print(est_psf_param)

## Debug

#### Matched with IDL output
- fourier_variable[ft(im)]
- image
- fourier_variable["ft(mean_object)"]

In [ ]:
# image (Matched IDL definition as it is directly copied from IDL)
print("fourier_variable[ft(im)]\n=============================")
utils.info(conv_image)

In [ ]:
# ft_im - matched the IDL output
ft_im = utils.fft2D(conv_image, DIMENSION)

print("fourier_variable[ft(im)]\n=============================")
utils.info(ft_im, imaginary = True)
plt.imshow(np.real(ft_im))

In [ ]:
# fourier_variable["ft(mean_object)"] - matched the IDL output
utils.info(amiral_child.fourier_variable["ft(mean_object)"], imaginary = True)
#plt.imshow(np.log10(np.abs(amiral_child.fourier_variable["ft(mean_object)"])+0.1))

In [ ]:
# psd_halo - matched the IDL output 
fXY = aosys.fX**2 + aosys.fY**2
utils.info(psd_halo)

In [ ]:
# psd - matched the IDL output 
utils.info(psd, imaginary = True)
plt.imshow(np.log10(psd))

In [ ]:
# otf_atmo - matched the IDL output
utils.info(otf_atmo, imaginary=True)
plt.imshow(otf_atmo)

#### In progress

For each variable, check the following: 
- max, min, mean, sum 
- sum(abs())


One thing 
- we know the otf_atmo is correct but the otf_tel is not correct due to the different definition. 

Question 

- We are using || for fourier variables, as long as the sum is the same, it should be fine? 

Current problem 

- I dont think we have the same defintion of fft and ifft, in order the have the same output, we must match all the defintions ... 

In [ ]:
# otf_tel (pup_tf in IDL)
# corr = np.fft.ifft2(pupil_plane)
# test = (np.abs(corr)**2) / np.sum(pupil_plane) * (DIMENSION**2)
# test is still correct
# pup_tf = fftshift(abs(fft(pup))^2,/INVERSE) /total(pup) * n_elements(pup) ; normalised diffraction pattern

utils.info(otf_tel, imaginary = True)
plt.imshow(np.log10(otf_tel))

Due to the error in otf_tel, the ft_psf is off too ... which means terms containing ft_psf is wrong ... 

In [ ]:
# fourier_variable["ft(psf)"] - the minimum is off, so as the sum
utils.info(amiral_child.fourier_variable["ft(psf)"], imaginary = True)
np.sum(np.abs(amiral_child.fourier_variable["ft(psf)"]))
plt.imshow(np.log10(np.abs(amiral_child.fourier_variable["psd(psf)"])))

In [ ]:
# psf - close! - because the imaginary part in the otf_tel is not correct ... 
utils.info(psf_total)
plt.imshow(np.log10(psf_total))

In [ ]:
#fourier_variable["error2"] |i - h*o_m|**2 
# minimum is off again, but the maximum and mean is good ...
# sum is 
utils.info(amiral_child.fourier_variable["error2"], imaginary = True)

# matched IDL output
np.sum(amiral_child.fourier_variable["error2"])
plt.imshow(np.log10(amiral_child.fourier_variable["error2"]))
print(np.sum(amiral_child.fourier_variable["error2"]))

In [ ]:
# fourier_variable["psd(psf)"] 
# mean is wrong
utils.info(amiral_child.fourier_variable["psd(psf)"], imaginary = True)

In [ ]:
# mean_error (tfi2_moy)
# Minimum is off but the rest matched the IDL ouput
mean_error = utils.mean_cir_array(amiral_child.fourier_variable["error2"])
utils.info(mean_error, imaginary = True)
np.sum(mean_error)

In [ ]:
# meam_ft_h2(tfh2_moy)
# mean and the minimum are off (given that the otf is off too ...)
meam_ft_h2 = utils.mean_cir_array(amiral_child.fourier_variable["psd(psf)"])
utils.info(meam_ft_h2, imaginary = True)
# sum is off
np.sum(meam_ft_h2)

In [ ]:
# sb_ini
# matched the IDL output

dimension = 512

sb_ini = (np.sum(amiral_child.fourier_variable["error2"][0,0:dimension]) + 
          np.sum(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]) 
          + np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, 0]) + 
            np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1])) / (4.*dimension - 4.)

print(sb_ini)

In [ ]:
# All off - length is correct
print("\nlen:", len(amiral_child.fourier_variable["error2"][0,0:dimension]))
utils.info(amiral_child.fourier_variable["error2"][0,0:dimension])
np.sum(amiral_child.fourier_variable["error2"][0,0:dimension]) - 1001557.7383473666

In [ ]:
# All off
print("\nlen:", len(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]))
utils.info(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension])
np.sum(amiral_child.fourier_variable["error2"][dimension-1, 0:dimension]) - 1002877.2089995614

In [ ]:
print("\nlen:", len(amiral_child.fourier_variable["error2"][1:dimension-1, 0]))
utils.info(amiral_child.fourier_variable["error2"][1:dimension-1, 0])
np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, 0]) - 1089953.2424355166

In [ ]:
print("\nlen:", len(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1]))
utils.info(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1])
np.sum(amiral_child.fourier_variable["error2"][1:dimension-1, dimension-1]) - 1086312.6571350598

In [ ]:
# k_ini
# almost matched IDL output (1.1213083933789605e+17)
k_ini = mean_error[1] 
k_ini - 1.1213083933789605e+17

In [ ]:
# mu_ini 
# matched IDL output
mu_ini = sb_ini / k_ini
print(mu_ini)
mu_ini - 1.8240768346243759e-14

Minimisation

In [ ]:
# check this again: amiral_child.fourier_variable["error2"]
arr = amiral_child.fourier_variable["error2"]
utils.info(arr)
np.sum(arr)

In [ ]:
# fourier_variable["rho"] 
# matched the IDL output
arr = amiral_child.fourier_variable["rho"]
utils.info(arr)
np.sum(arr)

In [ ]:
# fourier_variable["good_rho"]
arr = amiral_child.fourier_variable["good_rho"]
utils.info(arr)
np.sum(arr)

In [ ]:
# checking the initialisation - matched IDL ouput
amiral_child.hyperparam_initial(psf_guess)

In [ ]:
# fourier_variable["psd_object_ini"] - dspo
# - matched IDL output
# 1.686e-07   1.000e+00   1.152e-04     3.021e+01
arr = amiral_child.fourier_variable["psd_object_ini"]

plt.imshow(np.log10(arr))

utils.info(arr)
np.sum(arr)

In [ ]:
# fourier_variable["psd_model_i"] - denom
# matched the IDL output
# 1.824e-14   1.000e+00   4.300e-05 1.127e+01
arr = amiral_child.fourier_variable["psd_model_i"]
np.sum(arr)

plt.imshow(np.log10(arr))
utils.info(arr)
np.sum(arr)

In [ ]:
# fourier_variable["k_hat"] - does not matched ... due to tfh(?) 
arr = amiral_child.fourier_variable["k_hat"]
print(arr) # 9.791e+16

In [ ]:
# does not matched ...
test_k = amiral_child.fourier_variable["error2"] / amiral_child.fourier_variable["psd_model_i"]

utils.info(amiral_child.fourier_variable["error2"])
print(np.sum(arr))

test_sum = np.sum(test_k[amiral_child.fourier_variable["good_rho"]])
print(test_sum)

In [ ]:
# fourier_variable["psd_noise"] - doesnt not matched because it depends on k_hat
arr = amiral_child.fourier_variable["psd_noise"]
print(arr-1.786e+03)

In [ ]:
# fourier_variable["psd_object"] - dsp_objet
# doesnt not match ... 
# 1.651e+10   9.791e+16   1.128e+13    2.958e+18
arr = amiral_child.fourier_variable["psd_object"]
utils.info(arr)
print(np.sum(arr))

In [ ]:
# fourier_variable["psd_image_est"] - dsp_i
# doesnt match ... 
# 1.786e+03   9.791e+16   4.210e+12   1.104e+18
arr = amiral_child.fourier_variable["psd_image_est"]
utils.info(arr)
print(np.sum(arr))

In [ ]:
# criterion - crit
# does not match IDL output 
# 1654207.5616180671

print(value_criterion)

k_hat = 9.791e+16
psd_noise = mu_ini*k_hat
psd_object = k_hat*amiral_child.fourier_variable["psd_object_ini"]
psd_image_est = amiral_child.fourier_variable["psd(psf)"]*psd_object+psd_noise
good_rho = amiral_child.fourier_variable["good_rho"]
error = amiral_child.fourier_variable["error2"]

test_crit = 0.5 * np.sum(np.log(psd_image_est[good_rho])) + 0.5* np.sum(error[good_rho]/psd_image_est[good_rho])

In [ ]:
## gradient function in the minimiser
gradient = amiral_child.gradient(psf_guess)
print(gradient)

# IDL 
grad_psf: [313637.36820290622      -422776.16167403548      -20852.509443358213      -338842.43443541240       22386.038647048441]

grad_hyper: [-1.5547975389990738e+18       42539.024177912448      -111031.55773929584]

#### python
grad_psf:
[ 3.76661374e+05  1.76712368e-06  1.05124255e-04 -7.47756685e-03
  3.44429096e-04  0.00000000e+00  0.00000000e+00]
  
  
grad_hyper:
[5.82125344e+18
  3.72709536e+04 -9.05318129e+04]

## What to do now? 

1. Check the gradient function in python when bringing the analytical grad to the minimiser. Current gradient seems to be too large!

2. One thing to check is the defintion of the gradient 
    - Are they centred at the centre of the image or at the corner? 
    - Subtracting the wrong version will lead to an error

3. 
